## Load libraries

In [88]:
# Start with loading all necessary libraries
import numpy as np

import pandas as pd
from pandas.io.json import json_normalize

import json
from pprint import pprint as pp
print("Libraries loaded!")

Libraries loaded!


## Load dataset

In [2]:
# Create URL to CSV file (alternatively this can be a filepath)
filename = 'booking.csv/booking.csv'
#df = pd.read_csv(filename)
chunk = pd.read_csv(filename,chunksize=50)
df = pd.concat(chunk)
df.tail()

,booking_id,json_agg
4226,7583.0,"[{""id"":26798,""case_id"":1027,""type_id"":90,""book..."
4227,7585.0,"[{""id"":26794,""case_id"":613,""type_id"":94,""booki..."
4228,7587.0,"[{""id"":26803,""case_id"":1251,""type_id"":88,""book..."
4229,7592.0,"[{""id"":26800,""case_id"":1094,""type_id"":90,""book..."
4230,NaN,"[{""id"":107,""case_id"":1,""type_id"":88,""booking_i..."


In [24]:
# bookingJson=[]
# for i in range (1000):
#     bookingJson.append(json.loads(df.json_agg[i]))

In [141]:
# pp(bookingJson[900])

In [72]:
dfScripts=pd.DataFrame()
dfGeneral=pd.DataFrame()

In [142]:
# bookings

In [143]:
bookings=[]

for bookingNum in range (len(bookingJson)- 1, -1, -1):
    usedBookingKeys=['booking_id', 'case_id', 'create_date', 'type_id']
    usedScriptDiagnosisKeys=['description','pmb', 'code']    
    usedScriptKeys=['description','repeat', 'qty', 'dose', 'code', 'chronic','days_supplied']
    
    bookingData={}
    
    for b in range(len(bookingJson[bookingNum])):
        booking=bookingJson[bookingNum][b]
        if booking['description']=="SCRIPT" or booking['type_id']=='90':
            print("Script #", bookingNum)
            bookingData.update({"Doctor":booking['meta_data']['treating_doctor_id']})
            
            for usedBookingKey in usedBookingKeys:
                bookingData.update({usedBookingKey:booking[usedBookingKey]})

            stages=booking['stages']

            for n in range(len(stages)):
                for usedScriptKey in usedScriptKeys:
                    if usedScriptKey!="description" and usedScriptKey!="code":
                        bookingData[usedScriptKey]=list()
                    elif usedScriptKey=="description" :
                        bookingData["descriptionScript"]=list()
                    elif usedScriptKey=="code" :
                        bookingData["codeScript"]=list()
                        
                stageData=stages[n]['data']
                if 'script_item_list' in stageData:
                    scriptItems= stageData['script_item_list']
                      
                    for item in range(len(scriptItems)):
                        
                        for usedScriptKey in usedScriptKeys:
                            if usedScriptKey in scriptItems[item]:
                                if usedScriptKey!="description" and usedScriptKey!="code":
                                    val=bookingData[usedScriptKey]
                                    val.append(scriptItems[item][usedScriptKey])
                                    bookingData.update({usedScriptKey:val})
                                elif usedScriptKey=="description" :
                                    val=bookingData["descriptionScript"]
                                    val.append(scriptItems[item][usedScriptKey])
                                    bookingData.update({"descriptionScript":val})
                                elif usedScriptKey=="code":
                                    val=bookingData["codeScript"]
                                    val.append(scriptItems[item][usedScriptKey])
                                    bookingData.update({"codeScript":val})

                        for usedScriptDiagnosisKey in usedScriptDiagnosisKeys:
                            scriptDiagnosis=scriptItems[item]['diagnosis']
                            if len(scriptDiagnosis)>0:
                                if usedScriptDiagnosisKey in scriptDiagnosis[0]:
                                    bookingData.update({usedScriptDiagnosisKey:scriptDiagnosis[0][usedScriptDiagnosisKey]})
                        print("...................\n")  
                if 'patient' in stageData:
                    patientData= stageData['patient']
                    bookingData.update({'Gender':patientData['patient_gender']})
                    bookingData.update({'Birth Year':patientData['patient_dateofbirth'][:4]})      
            bookings.append(bookingData)
            print("---------------\n")
            print(bookingData)
            print("----------------\n\n")



Script # 999
...................

---------------

{'create_date': '2018-02-06T12:48:41.174575', 'Birth Year': '1995', 'descriptionScript': [], 'Doctor': 5, 'qty': [], 'booking_id': 2186, 'description': 'Unspecified parasitic disease', 'case_id': 843, 'days_supplied': [], 'repeat': [], 'chronic': [], 'Gender': 'F', 'pmb': False, 'dose': [], 'codeScript': [], 'type_id': 94, 'code': 'B89'}
----------------


Script # 998
...................

...................

---------------

{'create_date': '2018-02-06T13:23:26.108878', 'Birth Year': '1958', 'descriptionScript': [], 'Doctor': 4, 'qty': [], 'booking_id': 2185, 'description': 'Gastroenteritis and colitis of unspecified origin', 'case_id': 854, 'days_supplied': [], 'repeat': [], 'chronic': [], 'Gender': 'M', 'pmb': False, 'dose': [], 'codeScript': [], 'type_id': 90, 'code': 'A09.9'}
----------------


Script # 995
...................

---------------

{'create_date': '2018-02-06T12:41:00.021086', 'Birth Year': '1975', 'descriptionScript

KeyError: 'treating_doctor_id'

In [69]:
for bookingNum in range (len(bookingJson)- 1, -1, -1):

    for b in range(len(bookingJson[bookingNum])):
        booking=bookingJson[bookingNum][b]
        if booking['description']=="GENERAL EXAMINATION":
            print(booking['type_id'])
            print(booking["booking_id"])
            stages=booking['stages']
            for i in range(len(stages)):
                for key, value in stages[i].items():
                    if key=='data':
                            patientData=value
                            for k, v in patientData.items():
                                print("\n---------->",k,"\n")
                                if k=='jaccold':
                                    for element in range(len(v["items"])):
                                        print(v["items"][element]["description"],v["items"][element]["condition"])
                                elif k=='vital_measurements' or k=='sideroom_investigations' or k=='consultation':
                                    for VitalKey,VitalValue in v.items():
                                        if VitalKey=='urinedipstix':
                                            for UrineKey,UrineValue in VitalValue.items():
                                                print(UrineKey,UrineValue["selected_option"])
                                        else:
                                            print(VitalKey, VitalValue) 
                                elif k=='icd10':
                                    if len(v)>0:
                                        print('PMB', v[0]['pmb'])
                                        print('Code', v[0]['code'])
                                        print('Description', v[0]['description'])
                                elif k!='patient' and k!='systems' and key!='images':
                                    print(k,v) 
                                    print("...............................................")
                    elif key!='patient' and key!='images' and key!='finalized' and key!='triggerdone' and key!='description' and key!='depends' and key!='id':
                        print(key, value) 
                        print("----------------")
            

88
2185

----------> sideroom_investigations 

pro N/A
ery N/A
bil N/A
sg N/A
ph N/A
leu N/A
other None
nit N/A
ket N/A
ubg N/A
glu N/A
urate_test None
glucose None
fasting_glucose False

----------> images 

images []
...............................................

----------> vital_measurements 

blood_pressure_medicine None
immunizations_notes None
framingham 0
waist_hip_ratio 0
immunizations None
bmi 0
framingham_msg Systolic BP not filled in.

----------> jaccold 

Jaundice 
Anaemia (Pallor) 
Clubbing and Spooning 
Cyanosis 
Oedema 
Lymphadenopathy 
Dehydration 

----------> patient 


----------> systems 


----------> icd10 


----------> consultation 

quantity 1
description New and established patient: Co
code 0190

----------> consumables 

consumables []
...............................................

----------> general_notes 

general_notes 
...............................................

----------> procedures 

procedures []
...........................................

framingham_msg Systolic BP not filled in.

----------> jaccold 

Jaundice checked_and_normal
Anaemia (Pallor) checked_and_normal
Clubbing and Spooning checked_and_normal
Cyanosis checked_and_normal
Oedema checked_and_normal
Lymphadenopathy checked_and_normal
Dehydration checked_and_normal

----------> systems 


----------> icd10 

PMB False
Code L50.1
Description Idiopathic urticaria

----------> consultation 

quantity 1
description New and established patient: Co
code 0190

----------> consumables 

consumables []
...............................................

----------> general_notes 

general_notes 
...............................................

----------> procedures 

procedures []
...............................................

----------> clinical_procedures 

clinical_procedures []
...............................................
88
2168

----------> patient 


----------> complaint 

complaint Se sy sukkel met chroniese naarheid. Erge stress. Haar verhouding met die bio

ery N/A
bil N/A
sg N/A
ph N/A
leu N/A
other None
nit N/A
ket N/A
ubg N/A
glu N/A
urate_test None
glucose None
fasting_glucose False

----------> images 

images []
...............................................

----------> vital_measurements 

blood_pressure_medicine None
immunizations_notes None
framingham 0
waist_hip_ratio 0
immunizations None
bmi 0
framingham_msg Systolic BP not filled in.

----------> jaccold 

Jaundice checked_and_normal
Anaemia (Pallor) checked_and_normal
Clubbing and Spooning checked_and_normal
Cyanosis checked_and_normal
Oedema checked_and_normal
Lymphadenopathy checked_and_normal
Dehydration checked_and_normal

----------> systems 


----------> icd10 

PMB False
Code L23.2
Description Allergic contact dermatitis due to cosmetics

----------> consultation 

quantity 1
description New and established patient: Co
code 0190

----------> consumables 

consumables []
...............................................

----------> modifier_consultation 

modifier_con

code 0190

----------> consumables 

consumables []
...............................................

----------> general_notes 

general_notes 
...............................................

----------> procedures 

procedures []
...............................................

----------> clinical_procedures 

clinical_procedures []
...............................................
96
2149

----------> patient 


----------> complaint 

complaint BLWI
...............................................

----------> sideroom_investigations 

pro N/A
ery N/A
bil N/A
sg N/A
ph N/A
leu N/A
other None
nit N/A
ket N/A
ubg N/A
glu N/A
urate_test None
glucose None
fasting_glucose False

----------> images 

images []
...............................................

----------> vital_measurements 

blood_pressure_medicine None
immunizations_notes None
framingham 0
waist_hip_ratio 0
immunizations None
bmi 0
framingham_msg Systolic BP not filled in.

----------> jaccold 

Jaundice checked_and_normal

bmi 0
framingham_msg Systolic BP not filled in.

----------> jaccold 

Jaundice 
Anaemia (Pallor) 
Clubbing and Spooning 
Cyanosis 
Oedema 
Lymphadenopathy 
Dehydration 

----------> systems 


----------> icd10 

PMB False
Code R31
Description Unspecified haematuria

----------> consultation 

quantity 1
description New and established patient: Co
code 0190

----------> consumables 

consumables []
...............................................

----------> general_notes 

general_notes 
...............................................

----------> procedures 

procedures []
...............................................

----------> clinical_procedures 

clinical_procedures []
...............................................
88
2140

----------> patient 


----------> complaint 

complaint Haar Man se seun - Garry het haar gisteraand aangeval.
Hulle was in 'n vuisgeveg betrokke. Mekaar geslaan en gestamp.
Haar regter voorarm is nou seer. Keer sy houe en blou oor die regter voorarm.
.

...

Code N94.6
Description Dysmenorrhoea, unspecified

----------> consultation 

quantity 1
description New and established patient: Co
code 0190

----------> consumables 

consumables []
...............................................

----------> general_notes 

general_notes 
...............................................

----------> procedures 

procedures []
...............................................

----------> clinical_procedures 

clinical_procedures []
...............................................
88
1878

----------> sideroom_investigations 

pro N/A
ery N/A
bil N/A
sg N/A
ph N/A
leu N/A
other None
nit N/A
ket N/A
ubg N/A
glu N/A
urate_test None
glucose None
fasting_glucose False

----------> images 

images []
...............................................

----------> jaccold 

Jaundice checked_and_normal
Anaemia (Pallor) checked_and_normal
Clubbing and Spooning checked_and_normal
Cyanosis checked_and_normal
Oedema checked_and_normal
Lymphadenopathy checked_and_norm

...............................................

----------> vital_measurements 

blood_pressure_medicine None
immunizations_notes None
framingham 0
waist_hip_ratio 0
immunizations None
bmi 0
framingham_msg Systolic BP not filled in.

----------> systems 


----------> icd10 

PMB False
Code H60.9
Description Otitis externa, unspecified

----------> consultation 

quantity 1
description New and established patient: Co
code 0190

----------> consumables 

consumables []
...............................................

----------> modifier_consultation 

modifier_consultation {'quantity': 1}
...............................................

----------> general_notes 

general_notes 
...............................................

----------> procedures 

procedures []
...............................................

----------> clinical_procedures 

clinical_procedures []
...............................................
88
1844

----------> sideroom_investigations 

pro N/A
ery N/A
bil N/

ubg N/A
glu N/A
urate_test None
glucose None
fasting_glucose False

----------> images 

images []
...............................................

----------> jaccold 

Jaundice checked_and_normal
Anaemia (Pallor) checked_and_normal
Clubbing and Spooning checked_and_normal
Cyanosis checked_and_normal
Oedema checked_and_not_normal
Lymphadenopathy checked_and_normal
Dehydration checked_and_normal

----------> complaint 

complaint Chronic Medication required
...............................................

----------> vital_measurements 

blood_pressure_medicine None
immunizations_notes None
framingham 11
waist_hip_ratio 0
immunizations None
bmi 0
blood_pressure_diastolic 86
pulse 74
blood_pressure_systolic 128
framingham_msg Total cholestrol not filled in.

----------> systems 


----------> icd10 

PMB False
Code I10
Description Essential (primary) hypertension

----------> consultation 

quantity 1
description New and established patient: Co
code 0190

----------> consumables 

consu

pro N/A
ery N/A
bil N/A
sg N/A
ph N/A
leu N/A
other None
nit N/A
ket N/A
ubg N/A
glu N/A
urate_test None
glucose None
fasting_glucose False

----------> images 

images []
...............................................

----------> jaccold 

Jaundice 
Anaemia (Pallor) 
Clubbing and Spooning 
Cyanosis 
Oedema 
Lymphadenopathy 
Dehydration 

----------> vital_measurements 

blood_pressure_medicine None
immunizations_notes None
framingham 11
waist_hip_ratio 0
immunizations None
bmi 0
framingham_msg Systolic BP not filled in.

----------> systems 


----------> icd10 



KeyError: 'pmb'

## Train models

In [7]:
#svc = SVC()
#svc.fit(X_train, y_train)

#knn = KNeighborsClassifier()
#knn.fit(X_train, y_train) 

#dt = DecisionTreeClassifier()
#dt.fit(X_train, y_train) 

#nb= GaussianNB()
#nb.fit(X_train, y_train) 

## Predict

In [ ]:
#nb.predict(X_test)

In [ ]:
#nb.score(X_test, y_test)

In [ ]:
#cross_val_score(nb, X,y, cv=5)

In [ ]:
#np.mean(cross_val_score(nb, X, y, cv=5))